In [1]:
from IPython.display import clear_output
!pip install -U fsds_100719
clear_output()
from fsds_100719.imports import *


fsds_1007219  v0.7.22 loaded.  Read the docs: https://fsds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds_100719,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


[i] Pandas .iplot() method activated.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
data = pd.read_csv('product_review.csv',encoding= 'unicode_escape')

In [28]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [47]:
data.rename(columns={'tweet_text': 'tweet', 'emotion_in_tweet_is_directed_at': 'product', 'is_there_an_emotion_directed_at_a_brand_or_product':'emotion'}, inplace=True)

In [48]:
data.shape

(3291, 3)

In [49]:

data.dropna(inplace=True)

blanks = []  

for i,twt,prod,emo in data.itertuples():  
    if type(twt)==str:            
        if twt.isspace():         
            blanks.append(i)     
data.drop(blanks, inplace=True)

In [50]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/srn/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [51]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [52]:
data.head()

,tweet,product,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [53]:
data['scores'] = data['tweet'].apply(lambda tweet: sid.polarity_scores(tweet))

In [54]:
data.head()

,tweet,product,emotion,scores
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,"{'neg': 0.203, 'neu': 0.797, 'pos': 0.0, 'comp..."
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,"{'neg': 0.0, 'neu': 0.576, 'pos': 0.424, 'comp..."
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,"{'neg': 0.0, 'neu': 0.663, 'pos': 0.337, 'comp..."
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,"{'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp..."


In [55]:
data['positive'] = data['scores'].apply(lambda score_dict: score_dict['pos'])
data['negative'] = data['scores'].apply(lambda score_dict: score_dict['neg'])
data['neutral'] = data['scores'].apply(lambda score_dict: score_dict['neu'])
data['compound']= data['scores'].apply(lambda score_dict: score_dict['compound'])

In [69]:
data.head()

,tweet,product,emotion,scores,positive,negative,neutral,compound
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,"{'neg': 0.203, 'neu': 0.797, 'pos': 0.0, 'comp...",0.000,0.203,0.797,-0.6800
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,"{'neg': 0.0, 'neu': 0.576, 'pos': 0.424, 'comp...",0.424,0.000,0.576,0.9100
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.000,1.000,0.0000
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,"{'neg': 0.0, 'neu': 0.663, 'pos': 0.337, 'comp...",0.337,0.000,0.663,0.7269
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,"{'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp...",0.204,0.000,0.796,0.6249


In [67]:
data['emotion'].value_counts()

Positive emotion                      2672
Negative emotion                       519
No emotion toward brand or product      91
I can't tell                             9
Name: emotion, dtype: int64

In [77]:
bad_rows = data['emotion']== "I can't tell"

In [80]:
data = data[~bad_rows]

In [82]:
def emotion_p(c):
    if c >=0.2: 
        output = 'Positive emotion'
    elif c <=-0.2:
        output = 'Negative emotion'
    else:
        output = 'No emotion toward brand or product'
    return output
    

In [83]:
emotion_p(0)

'No emotion toward brand or product'

In [84]:
data['comp_score'] = data['compound'].apply(emotion_p)

In [85]:
data.head()

,tweet,product,emotion,scores,positive,negative,neutral,compound,comp_score
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,"{'neg': 0.203, 'neu': 0.797, 'pos': 0.0, 'comp...",0.000,0.203,0.797,-0.6800,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,"{'neg': 0.0, 'neu': 0.576, 'pos': 0.424, 'comp...",0.424,0.000,0.576,0.9100,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,0.000,1.000,0.0000,No emotion toward brand or product
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,"{'neg': 0.0, 'neu': 0.663, 'pos': 0.337, 'comp...",0.337,0.000,0.663,0.7269,Positive emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,"{'neg': 0.0, 'neu': 0.796, 'pos': 0.204, 'comp...",0.204,0.000,0.796,0.6249,Positive emotion


In [86]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [87]:
accuracy_score(data['emotion'],data['comp_score'])

0.5438756855575868

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
tfid = TfidfVectorizer()

In [90]:
from nltk import TweetTokenizer

In [91]:
tokenizer = TweetTokenizer()

In [92]:
tfid = TfidfVectorizer(tokenizer=tokenizer.tokenize)

In [95]:
tweets = tfid.fit_transform(data['tweet'])

In [97]:
tfid.inverse_transform(tweets)

[array(['#sxsw', 'stations', 'plugin', 'upgrade', 'to', 'need', '!',
        'dead', 'was', 'it', ',', '#rise_austin', 'at', 'tweeting', 'hrs',
        '3', 'after', 'iphone', '3g', 'a', 'have', 'i', '@wesley83', '.'],
       dtype='<U64'),
 array(['ts', 'free', 'giving', "they're", 'also', 'design', 'its', 'for',
        'appreciate', 'likely', "you'll", 'that', 'app', '/', 'ipad',
        'awesome', '?', '@fludapp', 'about', 'know', '@jessedee', '#sxsw',
        ',', 'at', 'iphone', '.'], dtype='<U64'),
 array(['down', 'them', 'sale', 'should', 'they', '2', '#ipad', 'wait',
        'not', 'can', '@swonderlin', 'also', 'for', '#sxsw', 'at', '.'],
       dtype='<U64'),
 array(['crashy', 'as', "isn't", 'festival', "year's", 'this', 'hope',
        '@sxsw', 'app', '#sxsw', 'iphone', 'i', '.'], dtype='<U64'),
 array(['wordpress', 'mullenweg', 'matt', '&', 'conferences', 'books',
        'tech', "o'reilly", 'tim', ')', 'google', '(', 'mayer', 'marissa',
        ':', 'fri', 'on', 'stuff', '